In [2]:
import function as fu
import numpy as np
import math
import pandas as pd

import matplotlib.pyplot as plt
import os
import tensorflow as tf
from keras import backend as K

from sklearn.metrics import confusion_matrix


In [35]:
#p_list = [0.37, 2.35, 9.08, 0.72, 7.67, 9.05, 7.15, 6.45, 8.47, 9.6]
p_list = [2,4,6,8,10]
comb = [[10,80], [20,50], [60,110], [70,100], [80,120], [80,140], [90,110], [100,120], [110,130], [150,180]]


for hardness in ['easy','mid']:
    if hardness == 'easy': c_01 = 2.3
    elif hardness == 'mid': c_01 = 3.6
        
    for N1,N2 in comb:
        list_con_mat = []
        list_p = []
        list_cost = []
        list_number = []
        list_precision = []
        list_recall = []
        list_f1 = []
        list_auc = []
        list_mcc = []
        list_gmean = []

        for p in p_list:
            con_mat_ = []
            weights_ = []
            p_ = []
            cost_ = []
            number_ = []
            precision_ = []
            recall_ = []
            f1_ = []
            auc_ = []
            mcc_ = []
            gmean_ = []
            
            for weight in np.linspace(1.5,3.0,16):
                list_con_mat_ = []
                list_p_ = []
                list_cost_ = []
                list_number_ = []
                list_precision_ = []
                list_recall_ = []
                list_f1_ = []
                list_auc_ = []
                list_mcc_ = []
                list_gmean_ = []
                
                for time in range(10):

                    con_mat,cost, number, precision, recall, f1, auc,mcc, gmean = fu.algorithm_spo(c_01,weight,hardness,p, N1,N2)

                    list_con_mat_.append(con_mat)
                    list_cost_.append(cost)
                    list_number_.append(number)
                    list_precision_.append(precision)
                    list_recall_.append(recall)
                    list_f1_.append(f1)
                    list_auc_.append(auc)
                    list_mcc_.append(mcc)
                    list_gmean_.append(gmean)
                    
                index = list_cost_.index(min(list_cost_))
                
                con_mat_.append(list_con_mat_[index])
                cost_.append(list_cost_[index])
                number_.append(list_number_[index])
                precision_.append(list_precision_[index])
                recall_.append(list_recall_[index])
                f1_.append(list_f1_[index])
                auc_.append(list_auc_[index])
                mcc_.append(list_mcc_[index])
                gmean_.append(list_gmean_[index])
                weight_.append(weight)
                
            index = cost_.index(min(cost_))
            
            con_mat = con_mat_[index]
            cost = cost_[index]
            number = number_[index]
            precision = precision_[index]
            recall = recall_[index]
            f1 = f1_[index]
            auc = auc_[index]
            mcc = mcc_[index]
            gmean = gmean_[index]
            weight = weights_[index]
                
                
            #print(f'initial_p:{p},cost:{cost},precision:{precision},recall:{recall},f1:{f1}')
            list_p.append(p)
            list_cost.append(cost)
            list_number.append(number)
            list_precision.append(precision)
            list_recall.append(recall)
            list_f1.append(f1)
            list_con_mat.append(con_mat)
            list_auc.append(auc)
            list_mcc.append(mcc)
            list_gmean.append(gmean)
            list_weight.append(weight)


        dict = {'p': list_p, 'num1':list_number, 'con_mat':list_con_mat, 'cost': list_cost, 'list_precision':list_precision, 'list_recall':list_recall, 'list_f1':list_f1,
               'list_auc': list_auc, 'list_mcc': list_mcc, 'list_gmean': list_gmean, 'list_weight': list_weight}   

        df = pd.DataFrame(dict) 
        method = 'spo'
        # saving the dataframe 
        df.to_csv('result/case0/{}_{}_{}_{}.csv'.format(method,hardness,N1,N2))

IndexError: list index out of range

In [29]:
df

,p,num1,con_mat,cost,list_precision,list_recall,list_f1,list_auc,list_mcc,list_gmean
0,2,23,"[888, 89, 12, 11]",332,0.48,0.11,0.18,0.55,0.19,0.329444


In [ ]:
def spo_plus_loss(c_01):
    c_01 = tf.cast(c_01, tf.float64)

    def my_loss(y_true,y_pred):
        y_pred = tf.cast(y_pred, tf.float64)
        y_true = tf.cast(y_true, tf.float64)
        
        max_1 = tf.cast(tf.math.maximum(2 * y_pred - 1,0),tf.float64)
        max_2 = tf.cast(tf.math.maximum(1-2 * y_pred, 0), tf.float64)
        
        loss =  y_pred*(1-y_pred) + max_1 * c_01 * y_true + (1- y_true) * max_2 + c_01 * y_true * (1-2 * y_pred) + (1 - y_true) * (2*y_pred - 1)
                                              
        return loss
    return my_loss
# ---------------------------------------------------------------------------------------------------------------

def Model_spo(c_01, X_train, y_train, X_test, y_test):
    
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape = (np.array(X_train).shape[-1],)),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss= spo_plus_loss(c_01))
    
    model.fit(X_train,y_train,shuffle = True, epochs= 30,verbose =0)
    y_pred = model.predict(X_test, batch_size=16, verbose =0)
 
    return y_pred

# ---------------------------------------------------------------------------------------------------------------

def algorithm_spo(c_01,hardness,p, N1,N2):
    
    if hardness =='easy':
        X_train, y_train, X_test, y_test = easy_data()
    elif hardness =='mid':
        X_train, y_train, X_test, y_test = mid_data()
        
    D_n = np.unique(y_test, return_counts = True)[1][0]
    D_p = np.unique(y_test, return_counts = True)[1][1]
        
    ticket3 = True
    while ticket3:    
        y_pred = Model_spo(c_01,X_train, y_train, X_test, y_test)
        fpr, fnr, thresholds = fpr_fnr(y_pred, y_test)

        if D_p <= N1:
            num = N1
        elif N2 <= D_p:
            num = N2
        elif N1 < D_p < N2:
            num = D_p

        th = sorted(y_pred, reverse = True)[num]
        y_result = np.where(y_pred>th, 1, 0)
        TN, FN, FP, TP = confusion_matrix(y_test, y_result).T.flatten()
        
        if not (TN, FN, FP, TP) == (D_n,D_p,0,0):
            ticket3 = False

    y_pred_sort_index = [b[0] for b in sorted(enumerate(y_pred),key=lambda i:i[1])]
    
    if (FP+TP) < N1:
        ticket1 = False
        ticket2 = False
        count = 0

        target = N1 - (FP+TP) 
        for i in range(len(y_result)):
            for j in range(len(y_result)):
                if y_pred_sort_index[j] == i: 
                    y_result[j] = 1
                    count += 1
            if count == target:
                ticket1 = True

            if ticket1 == True:
                break

    elif N2 < (FP+TP):
        ticket1 = False
        ticket2 = False
        count = 0

        target = (FP+TP) - N2
        for i in range(len(y_result)):
            for j in range(len(y_result)):
                if y_pred_sort_index[j] == i: 
                    y_result[j] = 1
                    count += 1
            if count == target:
                ticket2 = True

            if ticket2 == True:
                break

    TN, FN, FP, TP = confusion_matrix(y_test, y_result).T.flatten()
    cost = confusion_matrix(y_test, y_result).T[0][1] * c_01 + confusion_matrix(y_test, y_result).T[1][0] * c_10
    cost = round(cost)
 
    number = confusion_matrix(y_test, y_result).T[1][1] + confusion_matrix(y_test, y_result).T[1][0]       
    p = round(precision_score(y_test, y_result),2)
    r = round(recall_score(y_test, y_result),2)
    f = round(f1_score(y_test, y_result),2)
    auc = round(roc_auc_score(y_test, y_result),2)
    mcc = round(matthews_corrcoef(y_test, y_result),2)
    
    tpr = TP / (TP+FN)
    tnr = TN / (TN+FP)
    gmean = np.sqrt(tpr * tnr)
    
    con_mat = confusion_matrix(y_test, y_result).T.flatten().tolist()
    
    return con_mat,cost, number, p, r, f, auc,mcc, gmean


In [ ]:
#p_list = [0.37, 2.35, 9.08, 0.72, 7.67, 9.05, 7.15, 6.45, 8.47, 9.6]
p_list = [2,4,6,8,10]

comb = [(20,50),(80,100),(150,180)]
comb1 = [[80,120], [10,80], [80, 140]]
#
for hardness in ['easy','mid']:
    if hardness == 'easy': c_01 = 2.3
    elif hardness == 'mid': c_01 = 3.6
        
    for N1,N2 in comb1:
        list_con_mat = []
        list_p = []
        list_cost = []
        list_number = []
        list_precision = []
        list_recall = []
        list_f1 = []
        list_auc = []
        list_mcc = []
        list_gmean = []

        for p in p_list:
            ticket = True
            while ticket:
                con_mat,cost, number, precision, recall, f1, auc,mcc, gmean = fu.algorithm_spo(c_01,hardness,p, N1,N2)
                if not (con_mat[0] == 900) or (con_mat[-1] == 100): ticket = False
                
            #print(f'initial_p:{p},cost:{cost},precision:{precision},recall:{recall},f1:{f1}')
            list_p.append(p)
            list_cost.append(cost)
            list_number.append(number)
            list_precision.append(precision)
            list_recall.append(recall)
            list_f1.append(f1)
            list_con_mat.append(con_mat)
            list_auc.append(auc)
            list_mcc.append(mcc)
            list_gmean.append(gmean)


        dict = {'p': list_p, 'num1':list_number, 'con_mat':list_con_mat, 'cost': list_cost, 'list_precision':list_precision, 'list_recall':list_recall, 'list_f1':list_f1,
               'list_auc': list_auc, 'list_mcc': list_mcc, 'list_gmean': list_gmean}   

        df = pd.DataFrame(dict) 
        method = 'spo'
        # saving the dataframe 
        df.to_csv('result/case0/{}_{}_{}_{}.csv'.format(method,hardness,N1,N2))